In [54]:
#!pip install tensorflow

Proyecto compiado de: https://towardsdatascience.com/how-to-build-your-own-chatbot-using-deep-learning-bb41f970e281

In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
#keras preprocesamiento
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

Lee el json y crea los conjuntos de datos para entrenar el modelo

In [56]:

with open('../testFiles/intentsCV.json', encoding='utf-8') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [57]:
print(len(training_sentences))
print(len(training_labels))
print(len(labels))
print(len(responses))
responses

75
75
14
14


[['Hola, qué tal estas?', 'Hola', 'Qué tal?'],
 ['Adiós', 'Pasa un buen día', 'Espero que hablemos pronto'],
 ['De nada', 'Un placer'],
 ['Me gustaría guiarte a través del CV de Javier Aparicio',
  'Puedo hablarte de la experiendcia profesional de Javier Aparicio'],
 ['Mi nombre es Bot, Javi Bot',
  'Me llamo Javi Bot',
  'Puedes dirigirte a mi como Javi Bot'],
 ['Estoy formandome como Data Scientist',
  'Estoy estudiando  Machine Leraning'],
 ['Me falta del TFG para obtener el grado de Ingeniería Informática y, actualmente, estoy haciendo un Bootcamp sobre Data Science'],
 ['Ultimamente he estado en ANCERT y anteriormente estuve 14 años en el CSIC'],
 ['En ANCERT hice labores de analista programador VBA para análisis de datos y Java para backend'],
 ['En el CSIC hice todo tipo de labores informáticas, aunque sobre todo programar, tanto en VBA y Java, para el análisis de datos científicos'],
 ['Sí, tengo bastante experiencia',
  'Poseo gran conocimiento',
  'Lo he usado en diferentes p

In [58]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)
len(training_labels)
training_labels

array([ 8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  7,  7,  7,  7,  7,  7,
       13, 13, 13, 13, 13, 13, 13, 13,  0,  0,  0,  0,  0,  0,  9,  9,  9,
        9,  9, 10, 10, 10, 12, 12, 12,  5,  5,  5,  1,  1,  1,  4,  4,  4,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  6,  6,  6,  6,  3,
        3,  3,  3,  3,  2,  2,  2])

In [59]:
lbl_encoder.inverse_transform(training_labels)

array(['greeting', 'greeting', 'greeting', 'greeting', 'greeting',
       'greeting', 'greeting', 'greeting', 'greeting', 'greeting',
       'greeting', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye',
       'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks',
       'thanks', 'thanks', 'thanks', 'about', 'about', 'about', 'about',
       'about', 'about', 'name', 'name', 'name', 'name', 'name', 'now',
       'now', 'now', 'studies', 'studies', 'studies', 'experience',
       'experience', 'experience', 'ancert', 'ancert', 'ancert', 'csic',
       'csic', 'csic', 'skills', 'skills', 'skills', 'skills', 'skills',
       'skills', 'skills', 'skills', 'skills', 'skills', 'skills',
       'skills', 'future', 'future', 'future', 'future', 'createaccount',
       'createaccount', 'createaccount', 'createaccount', 'createaccount',
       'complaint', 'complaint', 'complaint'], dtype='<U13')

Tokenización de frases textuales con Tokenizer de keras

In [60]:
training_sentences

['Hi',
 'Hey',
 'Is anyone there?',
 'Hola',
 'Eo',
 'Hello',
 'Yep!',
 'Qué tal?',
 'Buenos días',
 'Buenas tardes',
 'Javi Bot',
 'Bye',
 'See you later',
 'Goodbye',
 'Adiós',
 'Hasta luego',
 'Hasta siempre',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Thanks for the help',
 'Gracias',
 'Ok',
 'Perfecto',
 'Muy bien',
 'Who are you?',
 'What are you?',
 'Who you are?',
 '¿En qué puedes ayudarme?',
 '¿qué haces?',
 '¿Para qué sirves?',
 'what is your name',
 'what should I call you',
 '¿Quién eres?',
 '¿Cómo te llamas?',
 'whats your name?',
 '¿Qué haces actualmente?',
 '¿Dónde estás ahora?',
 '¿Dónde trabajas?',
 'Formación',
 'Qué has esudiado?',
 'Tienes estudios superiores',
 '¿Qué has hecho?',
 '¿Dónde has trabajado?',
 '¿qué empresas?',
 'ANCERT',
 'Agencia Notarial de Certificación',
 'último trabajo',
 'CSIC',
 'Consejo Superior de Investigación Científica',
 'trabajo anterior',
 'JAVA',
 'Python',
 'SQL',
 'PHP',
 'VBA',
 'bases de datos',
 'Mysql',
 'Spring boot',
 'JPA'

Tratamiento de los textos para alimentar la red neuronal, Sería interesante probar una lematización en la tokenización

In [61]:
vocab_size = 1000 #número de palabras del vocabulario
embedding_dim = 16
max_len = 20 #tamaño del vector que representará al documento
oov_token = "<OOV>" #ayuda a identificar palabras que quedan fuera del vocabulario

tokenizer = Tokenizer()#crea la bolsa de palabras
tokenizer.fit_on_texts(training_sentences)#con las palabras de los documentos (oraciones)
word_index = tokenizer.word_index
#print(tokenizer.word_counts) #cuanto se repite una palabra
#print(tokenizer.document_count) #número de docs (frases)
print(tokenizer.word_index)#palabra, índice
#print(tokenizer.word_docs)#palabra índice de documento

sequences = tokenizer.texts_to_sequences(training_sentences) #transforma las frases a vectores de ínidces de palabras
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)# lleva las secuencias a vectores del mismo tamaño


{'you': 1, 'a': 2, 'qué': 3, 'to': 4, 'account': 5, '¿qué': 6, 'i': 7, 'de': 8, 'is': 9, 'are': 10, 'what': 11, 'te': 12, '¿dónde': 13, 'has': 14, 'trabajo': 15, 'create': 16, 'new': 17, 'complaint': 18, 'there': 19, 'hasta': 20, 'thanks': 21, 'for': 22, 'who': 23, 'haces': 24, 'your': 25, 'name': 26, 'el': 27, 'how': 28, 'open': 29, 'want': 30, 'an': 31, 'hi': 32, 'hey': 33, 'anyone': 34, 'hola': 35, 'eo': 36, 'hello': 37, 'yep': 38, 'tal': 39, 'buenos': 40, 'días': 41, 'buenas': 42, 'tardes': 43, 'javi': 44, 'bot': 45, 'bye': 46, 'see': 47, 'later': 48, 'goodbye': 49, 'adiós': 50, 'luego': 51, 'siempre': 52, 'thank': 53, "that's": 54, 'helpful': 55, 'the': 56, 'help': 57, 'gracias': 58, 'ok': 59, 'perfecto': 60, 'muy': 61, 'bien': 62, '¿en': 63, 'puedes': 64, 'ayudarme': 65, '¿para': 66, 'sirves': 67, 'should': 68, 'call': 69, '¿quién': 70, 'eres': 71, '¿cómo': 72, 'llamas': 73, 'whats': 74, 'actualmente': 75, 'estás': 76, 'ahora': 77, 'trabajas': 78, 'formación': 79, 'esudiado': 80,

In [62]:
sequences

[[32],
 [33],
 [9, 34, 19],
 [35],
 [36],
 [37],
 [38],
 [3, 39],
 [40, 41],
 [42, 43],
 [44, 45],
 [46],
 [47, 1, 48],
 [49],
 [50],
 [20, 51],
 [20, 52],
 [21],
 [53, 1],
 [54, 55],
 [21, 22, 56, 57],
 [58],
 [59],
 [60],
 [61, 62],
 [23, 10, 1],
 [11, 10, 1],
 [23, 1, 10],
 [63, 3, 64, 65],
 [6, 24],
 [66, 3, 67],
 [11, 9, 25, 26],
 [11, 68, 7, 69, 1],
 [70, 71],
 [72, 12, 73],
 [74, 25, 26],
 [6, 24, 75],
 [13, 76, 77],
 [13, 78],
 [79],
 [3, 14, 80],
 [81, 82, 83],
 [6, 14, 84],
 [13, 14, 85],
 [6, 86],
 [87],
 [88, 89, 8, 90],
 [91, 15],
 [92],
 [93, 94, 8, 95, 96],
 [15, 97],
 [98],
 [99],
 [100],
 [101],
 [102],
 [103, 8, 104],
 [105],
 [106, 107],
 [108],
 [109],
 [110, 8, 111],
 [112],
 [3, 12, 113, 114, 115, 27, 116],
 [117, 118, 119, 27, 120],
 [15, 121],
 [12, 122, 123, 124, 125],
 [7, 126, 4, 16, 2, 17, 5],
 [28, 4, 29, 2, 17, 5],
 [7, 30, 4, 16, 31, 5],
 [127, 1, 16, 31, 5, 22, 128],
 [28, 4, 29, 2, 17, 5],
 [129, 2, 18],
 [7, 30, 4, 130, 2, 18],
 [19, 9, 2, 18, 131, 2, 

Creación de la red neuronal: Secuentials

In [63]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 14)                238       
Total params: 16,782
Trainable params: 16,782
Non-trainable params: 0
_________________________________________________________________


In [64]:
epochs = 500 #número de veces en que todos los datos de entrenamiento pasan por la red en el proceso. Tip: aumentar el número de epoch hasta que la accuracy de los datos de validación (no los de entrenamiento) empiece a decrecer
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

4
Epoch 299/500
3/3 [==============================] - 0s 4ms/step - loss: 0.6275 - accuracy: 0.7773
Epoch 300/500
3/3 [==============================] - 0s 2ms/step - loss: 0.6658 - accuracy: 0.7895
Epoch 301/500
3/3 [==============================] - 0s 2ms/step - loss: 0.6196 - accuracy: 0.8079
Epoch 302/500
3/3 [==============================] - 0s 3ms/step - loss: 0.5928 - accuracy: 0.8614
Epoch 303/500
3/3 [==============================] - 0s 2ms/step - loss: 0.6161 - accuracy: 0.8302
Epoch 304/500
3/3 [==============================] - 0s 3ms/step - loss: 0.6158 - accuracy: 0.8223
Epoch 305/500
3/3 [==============================] - 0s 2ms/step - loss: 0.5647 - accuracy: 0.8380
Epoch 306/500
3/3 [==============================] - 0s 3ms/step - loss: 0.5824 - accuracy: 0.8380
Epoch 307/500
3/3 [==============================] - 0s 2ms/step - loss: 0.5891 - accuracy: 0.8184
Epoch 308/500
3/3 [==============================] - 0s 2ms/step - loss: 0.5733 - accuracy: 0.8274
Epoch 30

Salvo el modelo

In [65]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


In [2]:
model = keras.models.load_model('chat_model')

In [3]:
import pickle
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

    # load label encoder object
with open('label_encoder.pickle', 'rb') as enc:
    lbl_encoder = pickle.load(enc)

C:\Users\Javier\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [4]:
import json
with open('../testFiles/intentsCV.json', encoding='utf-8') as file:
    data = json.load(file)

In [5]:
import random
while True:
    print("User: ", end="")
    inp = input()
    if inp.lower() == "quit":
        break
    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                                truncating='post', maxlen=20))
    print([np.argmax(result)])
    tag = lbl_encoder.inverse_transform([np.argmax(result)])

    print(result)
    print(tag)

    for i in data['intents']:
        if i['tag'] == tag:
            print("ChatBot:", np.random.choice(i['responses']))

User: [8]
[[4.1531106e-03 3.2032635e-03 8.8032124e-08 2.6369065e-10 1.3542097e-08
  1.9321197e-07 1.1241958e-09 1.0516482e-03 9.8770463e-01 1.1781568e-04
  3.5863916e-08 2.7333850e-03 1.9191318e-06 1.0339305e-03]]
['greeting']
ChatBot: Qué tal?
User: [8]
[[4.1531106e-03 3.2032635e-03 8.8032124e-08 2.6369065e-10 1.3542097e-08
  1.9321197e-07 1.1241958e-09 1.0516482e-03 9.8770463e-01 1.1781568e-04
  3.5863916e-08 2.7333850e-03 1.9191318e-06 1.0339305e-03]]
['greeting']
ChatBot: Hola, qué tal estas?
User: [9]
[[6.3152584e-08 5.0635594e-03 7.4084935e-04 2.0944201e-04 7.0663952e-10
  3.3083424e-13 1.6524351e-03 2.0295415e-12 1.3558827e-04 9.9071538e-01
  4.9076581e-14 9.3682000e-11 8.1877958e-04 6.6386577e-04]]
['name']
ChatBot: Mi nombre es Bot, Javi Bot
User: [9]
[[5.6143184e-05 2.2183008e-01 5.3294911e-04 4.3928485e-05 1.8445227e-06
  8.0253448e-09 2.5672668e-03 1.3207576e-08 6.6685290e-03 6.7137206e-01
  1.2007340e-09 9.1648729e-07 2.0931048e-02 7.5995170e-02]]
['name']
ChatBot: Puedes 